In [2]:
# importo el archivo
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()

fname = "10_fiestaspopulares_festivales-datos_abiertos.csv"
df = pd.read_csv(fname, sep=";", encoding="latin1", engine="python")

print("Forma:", df.shape)
df.head(3)


Saving 10_fiestaspopulares_festivales-datos_abiertos.csv to 10_fiestaspopulares_festivales-datos_abiertos.csv
Forma: (3255, 27)


,cod_prov,cod_dep,cod_loc,Categoria,Provincia,Departamento,Localidad/Paraje,Nombre,Latitud,Longitud,...,Periodicidad,Año_primera_edicion,Ediciones,Mes de realización,Semana de realizacion,Duracion (días),Cantidad aprox de asistentes,Tipo_Entrada,Modalidad,último año de realizacion
0,6,6778,6778020.0,Fiesta y/o festival,Buenos Aires,San Vicente,San Vicente,Día Nacional del Payador,-3502403944,-5842059144,...,s/d,s/d,0,Julio,4,s/d,1000,Gratuita,s/d,2022
1,6,6497,6497060.0,Fiesta y/o festival,Buenos Aires,Lujan,Lujan,Festival del Pan Dulce en Luján,-3457065507,-5910954018,...,s/d,s/d,1,Diciembre,3,s/d,2000,Gratuita,s/d,2022
2,6,6196,6196010.0,Fiesta y/o festival,Buenos Aires,Coronel Pringles,Coronel Pringles,Pringles Cine Fest,-3798652106,-6135407157,...,s/d,s/d,1,Noviembre,2,s/d,250,Gratuita,s/d,2022


In [4]:
# Muestro por las columnas que esta formado el data set
print("Columnas del dataset cargado:")
for c in df.columns:
    print("-", c)


Columnas del dataset cargado:
- cod_prov
- cod_dep
- cod_loc
- Categoria
- Provincia 
- Departamento
- Localidad/Paraje
- Nombre
- Latitud
- Longitud
- Nota
- Fuente
- Temática_principal
- Temática_secundaria
- Aniversario
- Tipo de gestion
- Tipo de gestión privado
- Periodicidad
- Año_primera_edicion
- Ediciones
- Mes de realización
- Semana de realizacion
- Duracion (días)
- Cantidad aprox de asistentes
- Tipo_Entrada
- Modalidad
- último año de realizacion


In [21]:
import re, unicodedata
import matplotlib.pyplot as plt

#Normalizar nombres de columnas
def slugify(s: str) -> str:
    # quita tildes
    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    # minúsculas, espacios→_, barra→_, solo [a-z0-9_]
    s = s.strip().lower()
    s = s.replace('/', '_')
    s = re.sub(r'\s+', '_', s)
    s = re.sub(r'[^a-z0-9_]', '', s)
    return s

df.columns = [slugify(c) for c in df.columns]
print("Columnas normalizadas:", df.columns.tolist())

Columnas normalizadas: ['provincia', 'departamento', 'nombre', 'tematica_principal', 'tipo_de_gestion', 'mes_de_realizacion', 'duracion_dias', 'cantidad_aprox_de_asistentes', 'tipo_entrada', 'ultimo_ano_de_realizacion']


In [22]:
#Reemplazar marcadores de “sin dato” por NaN en todo el DF ----------
valores_no_disponibles = ['s/d','S/D','sd','SD','Sin dato','sin dato','SIN DATO','n/d','N/D']
df = df.replace(valores_no_disponibles, np.nan)

In [23]:
#Eliminar columnas poco útiles / muy faltantes ----------
cols_drop = [
    'cod_prov','cod_dep','cod_loc','categoria','Departamento', 'localidad_paraje',

'Nombre', 'latitud','longitud', 'nota','fuente',
    'tematica_secundaria','aniversario','tipo_de_gestion_privado','periodicidad','modalidad',
    'ano_primera_edicion','ediciones','aniversario',
    'semana_de_realizacion']
df = df.drop(columns=[c for c in cols_drop if c in df.columns], errors='ignore')
df.head(5)


,provincia,departamento,nombre,tematica_principal,tipo_de_gestion,mes_de_realizacion,duracion_dias,cantidad_aprox_de_asistentes,tipo_entrada,ultimo_ano_de_realizacion
0,Buenos Aires,San Vicente,Día Nacional del Payador,Tradicionales,Público,Julio,NaN,1000,Gratuita,2022
1,Buenos Aires,Lujan,Festival del Pan Dulce en Luján,Gastronómica,Público,Diciembre,NaN,2000,Gratuita,2022
2,Buenos Aires,Coronel Pringles,Pringles Cine Fest,Artística,Público,Noviembre,NaN,250,Gratuita,2022
3,Buenos Aires,General Alvarado,Fiesta del Kiwi: Mar y Sierras,Productiva,NaN,NaN,NaN,NaN,NaN,2022
4,Buenos Aires,Bragado,Suena Rock,Artística,NaN,Febrero,NaN,3000,Gratuita,2023


In [24]:
#corrijo el tipo de dato
cols_numericas = ['duracion_dias', 'ultimo_ano_de_realizacion', 'cantidad_aprox_de_asistentes']

for col in cols_numericas:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r'[^0-9.,]', '', regex=True)
            .str.replace(',', '.', regex=False)
            .replace('', np.nan)
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255 entries, 0 to 3254
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   provincia                     3255 non-null   object 
 1   departamento                  3255 non-null   object 
 2   nombre                        3255 non-null   object 
 3   tematica_principal            2892 non-null   object 
 4   tipo_de_gestion               2134 non-null   object 
 5   mes_de_realizacion            2333 non-null   object 
 6   duracion_dias                 1134 non-null   float64
 7   cantidad_aprox_de_asistentes  1166 non-null   float64
 8   tipo_entrada                  1751 non-null   object 
 9   ultimo_ano_de_realizacion     2146 non-null   float64
dtypes: float64(3), object(7)
memory usage: 254.4+ KB


In [29]:
# EXPORTO VERSIÓN BASE
df.to_csv("festivales_base_sin_imputar.csv", index=False)
print("✅ Archivo exportado correctamente: festivales_base_sin_imputar.csv")
print("Filas:", len(df))

# DESCARGO EL ARCHIVO
from google.colab import files
files.download("festivales_base_sin_imputar.csv")




✅ Archivo exportado correctamente: festivales_base_sin_imputar.csv
Filas: 3255


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
# --- IMPUTACIÓN DE DURACION (por provincia + departamento + temática) ---
df['duracion_dias'] = df['duracion_dias'].fillna(
    df.groupby(['provincia', 'departamento', 'tematica_principal'])['duracion_dias'].transform('mean')
)

# Si aún quedan nulos, usamos la media general
df['duracion_dias'] = df['duracion_dias'].fillna(df['duracion_dias'].mean())

print("✅ Nulos restantes en duracion_dias:", df['duracion_dias'].isna().sum())


✅ Nulos restantes en duracion_dias: 0


In [31]:
# --- IMPUTACIÓN DE ASISTENTES (por provincia + departamento + temática) ---
df['cantidad_aprox_de_asistentes'] = df['cantidad_aprox_de_asistentes'].fillna(
    df.groupby(['provincia', 'departamento', 'tematica_principal'])['cantidad_aprox_de_asistentes'].transform('mean')
)

# Si aún quedan nulos, usamos la media general
df['cantidad_aprox_de_asistentes'] = df['cantidad_aprox_de_asistentes'].fillna(df['cantidad_aprox_de_asistentes'].mean())

print("✅ Nulos restantes en cantidad_aprox_de_asistentes:", df['cantidad_aprox_de_asistentes'].isna().sum())


✅ Nulos restantes en cantidad_aprox_de_asistentes: 0


In [36]:
print(df[['duracion_dias', 'cantidad_aprox_de_asistentes']].isna().sum())
df['duracion_dias'] = df['duracion_dias' ].round(0).astype('Int64')
df['ultimo_ano_de_realizacion'] = df['ultimo_ano_de_realizacion'].round(0).astype('Int64')
df['cantidad_aprox_de_asistentes'] = df['cantidad_aprox_de_asistentes'].round(0).astype('Int64')
df.head(5)

duracion_dias                   0
cantidad_aprox_de_asistentes    0
dtype: int64


,provincia,departamento,nombre,tematica_principal,tipo_de_gestion,mes_de_realizacion,duracion_dias,cantidad_aprox_de_asistentes,tipo_entrada,ultimo_ano_de_realizacion
0,Buenos Aires,San Vicente,Día Nacional del Payador,Tradicionales,Público,Julio,3,1000,Gratuita,2022
1,Buenos Aires,Lujan,Festival del Pan Dulce en Luján,Gastronómica,Público,Diciembre,3,2000,Gratuita,2022
2,Buenos Aires,Coronel Pringles,Pringles Cine Fest,Artística,Público,Noviembre,3,250,Gratuita,2022
3,Buenos Aires,General Alvarado,Fiesta del Kiwi: Mar y Sierras,Productiva,NaN,NaN,3,11556,NaN,2022
4,Buenos Aires,Bragado,Suena Rock,Artística,NaN,Febrero,3,3000,Gratuita,2023


In [39]:
 #1) Normalizo  el nombre (sin tocar la columna original)
df['nombre_norm'] = df['nombre'].astype(str).str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)
# También limpio el mes textual, pero sigo en texto
df['mes_de_realizacion'] = df['mes_de_realizacion'].astype(str).str.strip()
df['mes_de_realizacion'] = df['mes_de_realizacion'].replace({'': np.nan, 'nan': np.nan})

# 2) Modo (mes más frecuente) por (provincia, nombre_norm)
modo_evento = (
    df.groupby(['provincia','nombre_norm'], dropna=False)['mes_de_realizacion']
      .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan)
      .reset_index(name='mes_ref'))

# 3) Completo los faltantes con la referencia del evento-provincia
df = df.merge(modo_evento, on=['provincia','nombre_norm'], how='left')
mask = df['mes_de_realizacion'].isna()
df.loc[mask, 'mes_de_realizacion'] = df.loc[mask, 'mes_ref']

# 4) (Opcional) fallback muy simple: modo por provincia
if df['mes_de_realizacion'].isna().any():
    modo_prov = (
        df.groupby('provincia')['mes_de_realizacion']
          .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan)
    )
    mask2 = df['mes_de_realizacion'].isna()
    df.loc[mask2, 'mes_de_realizacion'] = df.loc[mask2, 'provincia'].map(modo_prov)

# Limpieza de columnas auxiliares
df = df.drop(columns=['nombre_norm','mes_ref'])
df.head(15)


,provincia,departamento,nombre,tematica_principal,tipo_de_gestion,mes_de_realizacion,duracion_dias,cantidad_aprox_de_asistentes,tipo_entrada,ultimo_ano_de_realizacion,mes_num
0,Buenos Aires,San Vicente,Día Nacional del Payador,Tradicionales,Público,Julio,3,1000,Gratuita,2022,7.0
1,Buenos Aires,Lujan,Festival del Pan Dulce en Luján,Gastronómica,Público,Diciembre,3,2000,Gratuita,2022,12.0
2,Buenos Aires,Coronel Pringles,Pringles Cine Fest,Artística,Público,Noviembre,3,250,Gratuita,2022,11.0
3,Buenos Aires,General Alvarado,Fiesta del Kiwi: Mar y Sierras,Productiva,NaN,Noviembre,3,11556,NaN,2022,NaN
4,Buenos Aires,Bragado,Suena Rock,Artística,NaN,Febrero,3,3000,Gratuita,2023,2.0
5,Buenos Aires,La Plata,Carnaval Disidente,Expresiones del carnaval,Privado,Febrero,3,300,Gratuita,2023,2.0
6,Buenos Aires,General Pinto,FESTIVAL DE INVIERNO,Artística,Público,Julio,3,6000,Gratuita,2022,7.0
7,Buenos Aires,Mercedes,"Festival de Tango ""Rafael Rossi",Artística,Público,Marzo,3,7000,Gratuita,2023,3.0
8,Buenos Aires,Lomas de Zamora,Festival Cultura del Sur,Artística,Privado,Noviembre,3,529,Paga,2022,11.0
9,Buenos Aires,Moreno,FESTIVAL URBANO MORENO,Artística,Público,Octubre,3,50000,Gratuita,2022,10.0


In [40]:
# Mostrar cantidad de valores faltantes en mes_de_realizacion
faltan_mes = df['mes_de_realizacion'].isna().sum()
total = len(df)
print(f"Registros sin mes: {faltan_mes} de {total} ({faltan_mes/total:.2%})")


Registros sin mes: 0 de 3255 (0.00%)


In [41]:
df.to_csv("festivales_imputado.csv", index=False)
print("✅ Archivo exportado correctamente: festivales_imputado.csv")
from google.colab import files
files.download("festivales_imputado.csv")



✅ Archivo exportado correctamente: festivales_imputado.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
# Transformacion de variables a binarias
import pandas as pd

df = pd.read_csv("festivales_imputado.csv")
print("Columnas del dataset cargado:", df.columns.tolist())

#Eliminamos 2  Columnas que a partir de aqui son irrelevantes
cols_drop = ['nombre', 'departamento']
df = df.drop(columns=[c for c in cols_drop if c in df.columns], errors='ignore')
print("Columnas después de eliminar irrelevantes:", df.shape[1])

# pasamos a binarias Tipo_de_entrada:
df['tipo_entrada'] = df['tipo_entrada'].astype(str).str.lower().str.strip()
df['entrada_binaria'] = df['tipo_entrada'].map({'gratuita': 0, 'paga': 1})
# si alguna no encaja, se completa con 0 por defecto
df['entrada_binaria'] = df['entrada_binaria'].fillna(0)




# Hacemos ONE-HOT ENCODING a las variables categoricas
cat_cols = ['provincia', 'tematica_principal', 'tipo_de_gestion', 'mes_de_realizacion']
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Revisamos el resultado
print("Tamaño final del dataset codificado:", df_encoded.shape)
print("Primeras columnas:", df_encoded.columns[:10].tolist())

import numpy as np
import pandas as pd

# Aseguramos que todas las columnas con números se pasen a enteros reales
for col in df_encoded.columns:
    # Si es numérica o dummie, convertirla a int
    if pd.api.types.is_numeric_dtype(df_encoded[col]):
        df_encoded[col] = df_encoded[col].fillna(0).astype(int)

# Eliminamos la columna de texto 'tipo_entrada', ya que ahora usamos 'entrada_binaria'
if 'tipo_entrada' in df_encoded.columns:
    df_encoded = df_encoded.drop(columns=['tipo_entrada'])

# Confirmamos que no queden floats
print("Tipos de datos:")
print(df_encoded.dtypes.value_counts())




# Mostramos las primeras filas

df_encoded.head(10
        )


Columnas del dataset cargado: ['provincia', 'departamento', 'nombre', 'tematica_principal', 'tipo_de_gestion', 'mes_de_realizacion', 'duracion_dias', 'cantidad_aprox_de_asistentes', 'tipo_entrada', 'ultimo_ano_de_realizacion', 'mes_num']
Columnas después de eliminar irrelevantes: 9
Tamaño final del dataset codificado: (3255, 63)
Primeras columnas: ['duracion_dias', 'cantidad_aprox_de_asistentes', 'tipo_entrada', 'ultimo_ano_de_realizacion', 'mes_num', 'entrada_binaria', 'provincia_Catamarca', 'provincia_Chaco', 'provincia_Chubut', 'provincia_Ciudad Autónoma de Buenos Aires']
Tipos de datos:
int64    62
Name: count, dtype: int64


,duracion_dias,cantidad_aprox_de_asistentes,ultimo_ano_de_realizacion,mes_num,entrada_binaria,provincia_Catamarca,provincia_Chaco,provincia_Chubut,provincia_Ciudad Autónoma de Buenos Aires,provincia_Corrientes,...,mes_de_realizacion_Noviembre,mes_de_realizacion_Octubre,mes_de_realizacion_Septiembre,mes_de_realizacion_enero,mes_de_realizacion_febrero,mes_de_realizacion_julio,mes_de_realizacion_junio,mes_de_realizacion_marzo,mes_de_realizacion_noviembre,mes_de_realizacion_septiembre
0,3,1000,2022,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,2000,2022,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,250,2022,11,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,3,11556,2022,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,3,3000,2023,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,3,300,2023,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,3,6000,2022,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,7000,2023,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,3,529,2022,11,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,3,50000,2022,10,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [56]:
df_encoded.to_csv("festivales_codificado_modelo.csv", index=False, encoding="utf-8")
print("✅ Archivo exportado correctamente: festivales_icodificado_modelo.csv")
from google.colab import files
files.download("festivales_codificado_modelo.csv")

✅ Archivo exportado correctamente: festivales_icodificado_modelo.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>